### This file sets up the directories, imports py libs, runs push_and_build.sh, starts the session, does training and deployment

In [1]:
!mkdir opt
!mkdir opt/ml
!mkdir opt/ml/input
!mkdir opt/ml/input/data
!mkdir opt/ml/input/data/training
!mkdir opt/ml/model
!mkdir opt/ml/output
!mkdir opt/program

In [11]:
#!mkdir packages
#os.chdir('packages')
#!pip install tensorflow -t ./
#!pip install keras
#!pip install gunicorn  

In [3]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
import re
import boto3
import json


import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role
import os


In [9]:
os.getcwd()


'/home/ec2-user/SageMaker/race-vr/container'

In [5]:
! git status

On branch master
Your branch is up-to-date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   .ipynb_checkpoints/setup-checkpoint.ipynb
	modified:   setup.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [14]:
os.getcwd()

'/home/ec2-user/SageMaker/race-vr/container'

In [202]:
os.chdir('/home/ec2-user/SageMaker/race-vr/container')
! sh build_and_push.sh sagemaker-lstm

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  215.8MB
Step 1/8 : FROM ubuntu:16.04
 ---> 77be327e4b63
Step 2/8 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> f458b94b526e
Step 3/8 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy scipy==1.2.2 scikit-learn keras pandas flask gevent gunicorn tensorflow &&         rm -rf /root/.cache
 ---> Using cache
 ---> 90cd03d00a83
Step 4/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 8b3992bef087
Step 5/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> U

In [203]:
os.chdir('/home/ec2-user/SageMaker/race-vr/')

In [204]:
role = get_execution_role()
sess = sage.Session()
bucket = "sagemaker-wisdm2" #from get_data.ipynb
data_location = 's3://{}/data'.format(bucket)


In [205]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-lstm:latest'.format(account, region)


In [206]:
lstm = sage.estimator.Estimator(image,
                       role, 1, 'ml.m5.4xlarge',
                       output_path="s3://{}/output".format(bucket),
                       sagemaker_session=sess)

In [207]:
lstm.fit(data_location)

2020-04-01 19:39:56 Starting - Starting the training job...
2020-04-01 19:39:57 Starting - Launching requested ML instances......
2020-04-01 19:41:03 Starting - Preparing the instances for training...
2020-04-01 19:41:54 Downloading - Downloading input data
2020-04-01 19:41:54 Training - Downloading the training image......
2020-04-01 19:42:48 Training - Training image download completed. Training in progress.Using TensorFlow backend.
2020-04-01 19:42:49.522368: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-01 19:42:49.522441: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-01 19:42:49.522450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlop

In [211]:
#finally we export the parameters we need for deployment as json and upload it to s3
outjson = {'jobTrainingId': lstm._current_job_name, 'outputPath': "s3://{}/output".format(bucket), 'imageLocation':image}

with open("hyperParameters.json", "w") as json_file:
    json.dump(outjson, json_file)
    
    

In [212]:
os.chdir('/home/ec2-user/SageMaker/race-vr/')

In [213]:
bucket = "sagemaker-wisdm2"
data_location = 's3://{}'.format(bucket)

s3 = boto3.resource('s3')

s3.Bucket(bucket).upload_file("hyperParameters.json","hyperParameters.json" )



In [214]:
model_S3location = 'output/' + outjson['jobTrainingId'] + '/output/'
s3.Bucket(bucket).download_file(model_S3location + 'model.tar.gz', '/home/ec2-user/SageMaker/race-vr/opt/ml/model/model.tar.gz')


In [215]:
os.chdir('/home/ec2-user/SageMaker/race-vr/opt/ml/model/')
! tar -xf model.tar.gz

In [216]:
os.chdir('/home/ec2-user/SageMaker/race-vr/')

In [217]:
from sagemaker.predictor import csv_serializer
predictor = lstm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

---------------!

In [ ]:
shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)
# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

In [171]:
s3.Bucket(bucket).download_file('data/df0.csv', '/home/ec2-user/SageMaker/race-vr/df0.csv')

columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
test_data = pd.read_csv('df0.csv', header=0, names=columns)

In [180]:
test_data.head()

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164


In [190]:

def testPreprocess(df, timeSteps=400, step=40, testSize=0.2):
    '''preprocessing and split'''
    features=3
    RANDOM_SEED = 1992
    segments = []
    labels = []

    print('Preprocessing started')
    
    df = df.fillna(0)
    
    #normalise data otherwise loss= nan
    df['z-axis'] = df['z-axis'].astype(float)

    df['x-axis'] = (df['x-axis'] - df['x-axis'].min())/(df['x-axis'].max()-df['x-axis'].min())
    df['y-axis'] = (df['y-axis'] - df['y-axis'].min())/(df['y-axis'].max()-df['y-axis'].min())
    df['z-axis'] = (df['z-axis'] - df['z-axis'].min())/(df['z-axis'].max()-df['z-axis'].min())
    
    print( df['x-axis'].min(), df['y-axis'].min(), df['z-axis'].min())
    #redo it with  df.shape[0]%20 ==0 trying out different divisors (12 16 18 20 24) and a while to get to the largest

    #we reshape into 3d arrays of length evual to timesteps. final df is= (N*timesteps*3)
    for i in range(0, len(df) - timeSteps, step):
        xs = df['x-axis'].values[i: i + timeSteps]
        ys = df['y-axis'].values[i: i + timeSteps]
        zs = df['z-axis'].values[i: i + timeSteps]
        label = stats.mode(df['activity'][i: i + timeSteps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, timeSteps, features)
    labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

    print('Preprocessing completed')

    return train_test_split(
        reshaped_segments, labels, test_size=testSize, random_state=RANDOM_SEED)




In [191]:
X_train, X_test, y_train, y_test = testPreprocess(test_data)

Preprocessing started
0.0 0.0 0.0


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Preprocessing completed


In [246]:
test_data = X_train[0:10]

test_data.shape

(10, 400, 3)

In [248]:
print(predictor.predict(test_data))

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-lstm-2020-04-01-19-39-56-056 in account 264421419663 for more information.

In [ ]:
sess.delete_endpoint(predictor.endpoint)

In [238]:
test_data.shape

(1, 400, 3)

In [219]:
predictor.predict(test_data[0])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-lstm-2020-04-01-19-39-56-056 in account 264421419663 for more information.

## from quilt

In [ ]:
from sagemaker import Model
model = Model(
    model_data='s3://quilt-example/quilt/quilt_sagemaker_demo/model/sagemaker-demo-2019-01-18-22-01-23-156/output/model.tar.gz',
    image=image,
    role=role,
    sagemaker_session=sess
)
model.deploy(1, 'ml.m4.xlarge')

In [ ]:
predictor = sage.predictor.RealTimePredictor(
    'sagemaker-demo-2019-01-18-22-48-00-247', 
    sagemaker_session=sess, 
    content_type="text/csv")

In [ ]:
inp = "\n".join([",".join(l) for l in X_test.astype('str').tolist()])

In [ ]:
response = predictor.predict(inp)